In [ ]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download en_core_web_sm
#!pip install keras_preprocessing

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy
import pickle

In [ ]:
df=pd.read_csv('/content/ner_dataset.csv', encoding= 'unicode_escape')

In [ ]:
df['Sentence #']=df['Sentence #'].ffill()

In [ ]:
df=df[['Sentence #','Word','Tag']]
df.rename(columns={'Sentence #':'Sentence','Word':'Token','Tag':'POS'},inplace=True)

<ipython-input-46-e444fee1884a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Sentence #':'Sentence','Word':'Token','Tag':'POS'},inplace=True)


In [ ]:
df.head()

,Sentence,Token,POS
0,Sentence: 1,Thousands,O
1,Sentence: 1,of,O
2,Sentence: 1,demonstrators,O
3,Sentence: 1,have,O
4,Sentence: 1,marched,O


In [ ]:
# Group by 'Sentence' column and merge 'Token' and 'POS' columns separately
grouped_df = df.groupby('Sentence').agg({'Token': ' '.join, 'POS': ' '.join}).reset_index()
grouped_df.drop('Sentence',axis=1,inplace=True)

In [ ]:
grouped_df.head()

,Token,POS
0,Thousands of demonstrators have marched throug...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Iranian officials say they expect to get acces...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Helicopter gunships Saturday pounded militant ...,O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,They left after a tense hour-long standoff wit...,O O O O O O O O O O O
4,U.N. relief coordinator Jan Egeland said Sunda...,B-geo O O B-per I-per O B-tim O B-geo O B-gpe ...


In [ ]:
# Function to convert dataframe to spaCy format
def convert_df_to_spacy_format(df):
    spacy_data = []
    for idx, row in df.iterrows():
        text = row['Token']
        entities = []
        words = text.split()
        pos_tags = row['POS'].split()
        start_char = 0
        for word, pos_tag in zip(words, pos_tags):
            end_char = start_char + len(word)
            if pos_tag != 'O':
                entities.append((start_char, end_char, pos_tag))
            start_char = end_char + 1  # Account for the space after the word
        spacy_data.append((text, {'entities': entities}))
    return spacy_data

# Convert dataframe to spaCy format
spacy_data = convert_df_to_spacy_format(grouped_df)

In [ ]:
spacy_data[0]

('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
 {'entities': [(48, 54, 'B-geo'), (77, 81, 'B-geo'), (111, 118, 'B-gpe')]})

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [ ]:
for text, annot in tqdm(spacy_data):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot['entities']:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 47959/47959 [00:13<00:00, 3493.08it/s]


In [ ]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.52    0.58    1.06    0.40    0.01
  0     200        108.40   3530.78   60.58   62.19   59.05    0.61
  0     400        452.65   2445.96   72.51   72.87   72.15    0.73
  0     600        287.84   2231.36   75.89   76.38   75.41    0.76
  0     800        319.51   2738.15   78.33   79.65   77.05    0.78
  0    1000        346.24   3017.09   80.08   81.25   78.94    0.80
  0    1200        398.05   3574.76   81.97   83.29   80.70    0.82
  0    1400        487.46   4251.86   82.75   83.92   81.62    0.83
  0    1600        617.02   4906.31 

In [ ]:
!zip -r /content/file.zip /content/model-best
from google.colab import files
files.download("/content/file.zip")

  adding: content/model-best/ (stored 0%)
  adding: content/model-best/vocab/ (stored 0%)
  adding: content/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/model-best/vocab/lookups.bin (stored 0%)
  adding: content/model-best/vocab/vectors (deflated 45%)
  adding: content/model-best/vocab/strings.json (deflated 74%)
  adding: content/model-best/vocab/key2row (stored 0%)
  adding: content/model-best/tokenizer (deflated 81%)
  adding: content/model-best/tok2vec/ (stored 0%)
  adding: content/model-best/tok2vec/cfg (stored 0%)
  adding: content/model-best/tok2vec/model (deflated 7%)
  adding: content/model-best/ner/ (stored 0%)
  adding: content/model-best/ner/moves (deflated 79%)
  adding: content/model-best/ner/cfg (deflated 33%)
  adding: content/model-best/ner/model (deflated 9%)
  adding: content/model-best/config.cfg (deflated 61%)
  adding: content/model-best/meta.json (deflated 69%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>